<a href="https://colab.research.google.com/github/slayerzeroa/Railway_Accident_BERT/blob/main/Tagtog_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 라이브러리
import torch
from torch import nn, optim, tensor
from torch.utils.data import DataLoader, Dataset,TensorDataset, random_split
import torch.nn.functional as F

import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import accuracy_score, mean_squared_error

# from transformers import BertTokenizer

import json
import glob
import re
import os
import ast
import html

import pandas as pd

from google.colab import files

In [2]:
import os
import requests

#로그인 위치
url = "https://tagtog.com/-login"

#다운로드 위치
file_url = 'https://www.tagtog.com/slayerzeroa/Railway_BERT/-downloads/dataset-as-anndoc' 
o_file = 'Railway_Annotation.zip'  
if os.path.exists(o_file):
    os.remove(o_file)

#로그인 정보
login_info = {
    'loginid' : 'slayerzeroa@gmail.com',
    'password' : 'gyysxw5rU3NzrYX',
}

#로그인
with requests.Session() as s:
    login_req = s.post(url, data=login_info)
    r = s.get(file_url)

    with open(o_file,"wb") as output:
        output.write(r.content)

In [3]:
#압축파일 풀기
import zipfile
import shutil
folder_path = "./tagtog_relation_extraction/"

zip_ = zipfile.ZipFile("Railway_Annotation.zip")
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
zip_.extractall(folder_path)

In [10]:
import json
import glob
import re
import os
#폴더 경로
folder_name = "./tagtog_relation_extraction/Railway_BERT/"

#context list   # context == plain.html
context_name_list = os.listdir(folder_name + "plain.html/pool")
print(context_name_list)

#relation 폴더 경로   # relation == ann.json
relation_folder_paths = glob.glob(folder_name + "ann.json/master/pool")

#context 폴더 경로
# contexts_folders_paths = glob.glob(folder_name + "plain.html/pool/*")
contexts_folders_paths = [folder_name + "plain.html/pool/" + c for c in context_name_list]
print(contexts_folders_paths)

#anntation_lenged 정보
annotations_legend = folder_name + "annotations-legend.json"
with open(annotations_legend,"r") as f:
    annotations_legend = json.load(f)


['a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf.plain.html']
['./tagtog_relation_extraction/Railway_BERT/plain.html/pool/a3DnzcVSvJKG25OeznRpiW5zLgi8-sample.pdf.plain.html']


# TagTog Annotation json 전처리

In [11]:
def split_relation_start(text):
  text = text.split('|')[-1]
  start = text.split(',')[0]
  start = int(start)
  return start

In [21]:
for context_name, relation_folder, contexts_folder in zip(context_name_list, relation_folder_paths, contexts_folders_paths):
    # relation files와 context files 리스트 출력
    file_ids = [file_name for file_name in os.listdir(relation_folder)]
    file_nums = [ids.split("-")[1] for ids in file_ids]
    relation_files = [relation_folder + "/"+ file_id for file_id in file_ids]
    context_files = [contexts_folder for file_id in file_ids]

    for relation_file, context_file, file_num in zip(relation_files, context_files, file_nums):
      with open(relation_file,"r") as f:
        relation = json.load(f)

        idx_list = []
        classid_list = []
        part_list = []
        text_list = []
        start_list = []

        for idx, content in enumerate(relation['entities']):
          idx_list.append(idx)
          classid_list.append(content['classId'])
          part_list.append(content['part'])
          text_list.append(content['offsets'][0]['text'])
          start_list.append(content['offsets'][0]['start'])

        df = pd.DataFrame(zip(idx_list, start_list, classid_list, part_list, text_list))
        df.columns = ['Index', 'Start','Class', 'Part', 'Text']

        doc_list = []
        loc_list = []
        cls_list = []
        part_list = []
        subject_list = []
        object_list = []
        label_list = []

        for rel in relation['relations']:
          subject = split_relation_start(rel['entities'][0])
          subject_class = df.loc[df['Start']==subject]['Class'].values[0]
          subject_part = df.loc[df['Start']==subject]['Part'].values[0]
          subject_text = df.loc[df['Start']==subject]['Text'].values[0]
          

          object = split_relation_start(rel['entities'][1])
          object_class = df.loc[df['Start']==object]['Class'].values[0]
          object_part = df.loc[df['Start']==object]['Part'].values[0]
          object_text = df.loc[df['Start']==object]['Text'].values[0]

          loc = str(subject) + '_' + str(object)
          cls = subject_class + '_' + object_class
          part = subject_part + '_' + object_part

          doc_list.append(str(file_nums)[2:-11])
          loc_list.append(loc)
          cls_list.append(cls)
          part_list.append(part)
          subject_list.append(subject_text)
          object_list.append(object_text)
          label_list.append(1)

        Rel_df = pd.DataFrame(zip(doc_list, loc_list, cls_list, part_list, subject_list, object_list, label_list))
        Rel_df.columns = ['Doc', 'Loc', 'Cls', 'Part', 'Sub', 'Obj', 'Label']

In [22]:
print(Rel_df)

          Doc          Loc       Cls       Part  \
0  sample.pdf    3704_3607   e_6_e_4  s1p5_s1p5   
1  sample.pdf    3741_3607   e_6_e_4  s1p5_s1p5   
2  sample.pdf  13057_12960   e_6_e_4  s1p6_s1p6   
3  sample.pdf  13094_12960   e_6_e_4  s1p6_s1p6   
4  sample.pdf    3394_3607  e_22_e_4  s1p5_s1p5   
5  sample.pdf    3434_3607  e_22_e_4  s1p5_s1p5   
6  sample.pdf    3455_3607  e_22_e_4  s1p5_s1p5   
7  sample.pdf  13368_13094  e_23_e_6  s1p6_s1p6   
8  sample.pdf  13525_13057  e_23_e_6  s1p6_s1p6   

                                                 Sub  \
0                화차에 컨테이너를 싣고 내릴 때 차축 베어링에 충격이 가해진 것   
1  차축 베어링의 분해검수 주기와 교환 주기가 화차에 적재되는 화 물의 종류, 적재방법...   
2                화차에 컨테이너를 싣고 내릴 때 차축 베어링에 충격이 가해진 것   
3  차축 베어링의 분해검수 주기와 교환 주기가 화차에 적재되는 화 물의 종류, 적재방법...   
4                뒷 대차 첫 번째 차축과 두 번째 차축이 진행방향 좌측으로 탈선   
5                                    좌 측 세 번째 차축이 절손   
6                    탈선화차와 14번째 화차 사이의 공기호스 가 빠져있는 것   
7  화차 차축 베어링의 분해검수 주기와 교환 주기를 화차에 적재되는 화 

In [23]:
print(type(annotations_legend))

doc_name = 'sample'
with open("annotations-legend.json","r") as f:
    annotations_legend = json.load(f)

print(annotations_legend)

<class 'dict'>


FileNotFoundError: ignored

In [ ]:
with open("sample.ann.json","r") as f:
    relation = json.load(f)

print(relation['annotatable'])
print(relation['anncomplete'])
print(len(relation['entities']))

idx_list = []
classid_list = []
part_list = []
text_list = []
start_list = []

for idx, content in enumerate(relation['entities']):
  idx_list.append(idx)
  classid_list.append(content['classId'])
  part_list.append(content['part'])
  text_list.append(content['offsets'][0]['text'])
  start_list.append(content['offsets'][0]['start'])

df = pd.DataFrame(zip(idx_list, start_list, classid_list, part_list, text_list))
df.columns = ['Index', 'Start','Class', 'Part', 'Text']

print(df)

{'parts': ['s1p1', 's1p2', 's1p3', 's1p4', 's1p5', 's1p6', 's1p7']}
False
14
    Index  Start Class  Part  \
0       0   3394  e_22  s1p5   
1       1   3434  e_22  s1p5   
2       2   3455  e_22  s1p5   
3       3   3607   e_4  s1p5   
4       4   3704   e_6  s1p5   
5       5   3741   e_6  s1p5   
6       6  12960   e_4  s1p6   
7       7  13057   e_6  s1p6   
8       8  13094   e_6  s1p6   
9       9  13368  e_23  s1p6   
10     10  13525  e_23  s1p6   
11     11  13608  e_23  s1p6   
12     12  13689  e_23  s1p6   
13     13  13763  e_23  s1p6   

                                                 Text  
0                 뒷 대차 첫 번째 차축과 두 번째 차축이 진행방향 좌측으로 탈선  
1                                     좌 측 세 번째 차축이 절손  
2                     탈선화차와 14번째 화차 사이의 공기호스 가 빠져있는 것  
3   차축에 설치되어 있는 차축 베어링 외륜의 내면과 롤러에 결함이 있는 상태에서 운행 ...  
4                 화차에 컨테이너를 싣고 내릴 때 차축 베어링에 충격이 가해진 것  
5   차축 베어링의 분해검수 주기와 교환 주기가 화차에 적재되는 화 물의 종류, 적재방법...  
6   차축에 설치되어 있는 차축 베어링 외륜의 내면과 롤러에 결함이 있는 상태에서 운행 

In [ ]:
print(relation['relations'])

[{'classId': 'r_19', 'type': 'linked', 'directed': False, 'entities': ['s1p5|e_6|3704,3738', 's1p5|e_4|3607,3683'], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}}, {'classId': 'r_19', 'type': 'linked', 'directed': False, 'entities': ['s1p5|e_6|3741,3820', 's1p5|e_4|3607,3683'], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}}, {'classId': 'r_19', 'type': 'linked', 'directed': False, 'entities': ['s1p6|e_6|13057,13091', 's1p6|e_4|12960,13036'], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}}, {'classId': 'r_19', 'type': 'linked', 'directed': False, 'entities': ['s1p6|e_6|13094,13173', 's1p6|e_4|12960,13036'], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}}, {'classId': 'r_24', 'type': 'linked', 'directed': False, 'entities': ['s1p5|e_22|3394,3428', 's1p5|e_4|3607,3683'], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}}, {'classId': 'r_24', 

In [ ]:
print(annotations_legend)
print(len(annotations_legend))

{'e_23': 'SOLUTION', 'r_19': 'mz4f90xj280(e_6|e_4)', 'r_26': '5zjoi2gg5ps(e_6|e_23)', 'e_22': 'ACCIDENT', 'r_25': '6oyjvcblqfe(e_22|e_23)', 'e_6': 'IMPLIED', 'r_27': 'f19f8drqjkn(e_4|e_23)', 'e_4': 'CAUSE', 'r_24': 'yz4ctet7cda(e_4|e_22)'}
9


In [ ]:
for rel in relation['relations']:
  print(rel)

{'classId': 'r_19', 'type': 'linked', 'directed': False, 'entities': ['s1p5|e_6|3704,3738', 's1p5|e_4|3607,3683'], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}}
{'classId': 'r_19', 'type': 'linked', 'directed': False, 'entities': ['s1p5|e_6|3741,3820', 's1p5|e_4|3607,3683'], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}}
{'classId': 'r_19', 'type': 'linked', 'directed': False, 'entities': ['s1p6|e_6|13057,13091', 's1p6|e_4|12960,13036'], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}}
{'classId': 'r_19', 'type': 'linked', 'directed': False, 'entities': ['s1p6|e_6|13094,13173', 's1p6|e_4|12960,13036'], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}}
{'classId': 'r_24', 'type': 'linked', 'directed': False, 'entities': ['s1p5|e_22|3394,3428', 's1p5|e_4|3607,3683'], 'confidence': {'state': 'pre-added', 'who': ['user:slayerzeroa'], 'prob': 1}}
{'classId': 'r_24', 'type'

In [ ]:
def split_relation_start(text):
  text = text.split('|')[-1]
  start = text.split(',')[0]
  start = int(start)
  return start

In [ ]:
doc_list = []
loc_list = []
cls_list = []
part_list = []
subject_list = []
object_list = []
label_list = []
'''
Loc : Sub, Obj 위치
Cls : Sub, Obj 클래스
Part : Sub, Obj html 상 문단
Sub : Sub 내용
Obj : Obj 내용
Label : 관계 있으면 1 없으면 0
Relation DataFrame Content
'''
for rel in relation['relations']:
  subject = split_relation_start(rel['entities'][0])
  subject_class = df.loc[df['Start']==subject]['Class'].values[0]
  subject_part = df.loc[df['Start']==subject]['Part'].values[0]
  subject_text = df.loc[df['Start']==subject]['Text'].values[0]
  

  object = split_relation_start(rel['entities'][1])
  object_class = df.loc[df['Start']==object]['Class'].values[0]
  object_part = df.loc[df['Start']==object]['Part'].values[0]
  object_text = df.loc[df['Start']==object]['Text'].values[0]

  loc = str(subject) + '_' + str(object)
  cls = subject_class + '_' + object_class
  part = subject_part + '_' + object_part

  doc_list.append(doc_name)
  loc_list.append(loc)
  cls_list.append(cls)
  part_list.append(part)
  subject_list.append(subject_text)
  object_list.append(object_text)
  label_list.append(1)

Rel_df = pd.DataFrame(zip(doc_list, loc_list, cls_list, part_list, subject_list, object_list, label_list))
Rel_df.columns = ['Doc', 'Loc', 'Cls', 'Part', 'Sub', 'Obj', 'Label']

In [ ]:
print(Rel_df)

      Doc          Loc       Cls       Part  \
0  sample    3704_3607   e_6_e_4  s1p5_s1p5   
1  sample    3741_3607   e_6_e_4  s1p5_s1p5   
2  sample  13057_12960   e_6_e_4  s1p6_s1p6   
3  sample  13094_12960   e_6_e_4  s1p6_s1p6   
4  sample    3394_3607  e_22_e_4  s1p5_s1p5   
5  sample    3434_3607  e_22_e_4  s1p5_s1p5   
6  sample    3455_3607  e_22_e_4  s1p5_s1p5   
7  sample  13368_13094  e_23_e_6  s1p6_s1p6   
8  sample  13525_13057  e_23_e_6  s1p6_s1p6   

                                                 Sub  \
0                화차에 컨테이너를 싣고 내릴 때 차축 베어링에 충격이 가해진 것   
1  차축 베어링의 분해검수 주기와 교환 주기가 화차에 적재되는 화 물의 종류, 적재방법...   
2                화차에 컨테이너를 싣고 내릴 때 차축 베어링에 충격이 가해진 것   
3  차축 베어링의 분해검수 주기와 교환 주기가 화차에 적재되는 화 물의 종류, 적재방법...   
4                뒷 대차 첫 번째 차축과 두 번째 차축이 진행방향 좌측으로 탈선   
5                                    좌 측 세 번째 차축이 절손   
6                    탈선화차와 14번째 화차 사이의 공기호스 가 빠져있는 것   
7  화차 차축 베어링의 분해검수 주기와 교환 주기를 화차에 적재되는 화 물의 종류, 적...   
8  컨테이너를 화차에서 들어 올릴 때 화차가

#Google Spreadsheet 연동

In [ ]:
!pip install gspread
!pip install oauth2client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
key = files.upload()

Saving railway-tagtog-d24e6901ddb0.json to railway-tagtog-d24e6901ddb0.json


In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

#json key file 위치
json_file_name = './railway-tagtog-d24e6901ddb0.json'

# json key file을 이용하여 접속
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file_name, scope)
gc = gspread.authorize(credentials)

#구글 스프레드 시트 주소
spreadsheet_url = "https://docs.google.com/spreadsheets/d/1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo/edit#gid=0"

# 스프레드시트 문서 가져오기
doc = gc.open_by_url(spreadsheet_url)
## gc.create(spreadsheet_name) # 스프레드시트 생성


#시트 선택하기
sheet_name = "annotation"
worksheet = doc.worksheet(sheet_name) #해당 시트가 있는 경우 불러오기
## 403 error가 뜰 경우, google sheets API를 활성시켜줘야 함

In [ ]:
# 시트의 모든 데이터 가져오기
values = worksheet.get_all_values()
header, rows = values[0], values[1:]
data = pd.DataFrame(rows, columns=header)
column_list = ['Doc', 'Loc', 'Cls', 'Part', 'Sub', 'Obj', 'Label']
data = data[column_list]
data.head()

# 워크시트 범위 설정
worksheet.resize(len(doc_list)+1,10)
list_range = f"a2:g{len(doc_list)+1}"
cell_list = worksheet.range(list_range)

# 내용 작성
for i in range(len(cell_list)//len(column_list)):
    cell_list[(7*i)].value = doc_list[i]
    cell_list[(7*i)+1].value = loc_list[i]
    cell_list[(7*i)+2].value = cls_list[i]
    cell_list[(7*i)+3].value = part_list[i]
    cell_list[(7*i)+4].value = subject_list[i]
    cell_list[(7*i)+5].value = object_list[i]
    cell_list[(7*i)+6].value = label_list[i]
    
worksheet.update_cells(cell_list)

{'spreadsheetId': '1cpt2ad2pw0N2tC1H3MPtRAaZzFmgCh2If0gHOQKEIIo',
 'updatedRange': 'annotation!A2:G10',
 'updatedRows': 9,
 'updatedColumns': 7,
 'updatedCells': 63}